In [1]:
import os 
import re
import sys
import numpy as np
import pandas as pd
# import nltk
import tensorflow as tf

In [2]:
new_filename = "28_01_2020_NewData.txt"

#read eng sentences

        

In [17]:
def prepare_tag_data(filename):
    eng_lines = []
    with open(filename,'r',encoding='utf-8') as f:
        for line in f:
            line = line.strip().replace('<s>','').replace('</t>','').split('</s> <t>')
            line = line[0].strip()
            line = re.sub('\s+',' ',line)
            eng_lines.append(line)
    return eng_lines

lns = prepare_tag_data(filename=new_filename)

In [18]:
len(lns),lns[:2]

(800640,
 ['rendered Poll library raise MLSDFIELD void ferrari bits dan rangers of periodic explore register load MLSDREG shortcuts quiet until mason attachment cleared relations',
  'Poll MLSDFIELD bits of register MLSDREG until cleared'])

In [5]:
#build params (Hyperparameters)

In [6]:
config = {}
EPOCHS = 15
BATCH_SIZE = 32
LR = 0.001
DECAY_LR = 0.9
OPTM = "Adam"
DROPOUT = 0.3
EPOCHS_NO_IMPROVEMENT = 4
HIDDEN_DIM = 256
DATA_FILEPATH = "data_folder/data.txt"
GLOVE_FILEPATH = "glove/glove.6B.100d.txt"
VOCAB_FILENAME = "data_folder/vocabs.txt"
TAG_FILENAME = "data_folder/tags.txt"



In [7]:
#build files from data

In [8]:
def data_batches(data,batch_size=32):
    """Args >> data=tuple(sentence,tag)
    returns data in batches"""
    steps = len(data)//batch_size
    x_batch, y_batch = [], []
    for (x,y) in data:
        if len(x_batch) == batch_size:
            yield x_batch, y_batch
            x_batch, y_batch = [], []
        if type(x[0])==tuple:
            x = zip(*x)
            x_batch += [x]
            y_batch += [y]
    if len(x_batch) != 0:
        yield x_batch, y_batch

In [9]:
UNK = "$UNK$"
NUM = "$NUM$"
NONE = "O"

In [10]:
def get_vocabs(datasets):
    """Building vocabs present in datasets"""
    print("Building Vocab ....")
    vocab_words = set()
    vocab_tags = set()
    for dataset in datasets:
        for words, tags in dataset:
            vocab_words.update(words)
            vocab_tags.update(tags)
    print("-done vocab words {s}--- vocab_tags {d}".format(s=len(vocab_words),d=len(vocab_tags)))
    return vocab_words, vocab_tags

In [11]:
def get_glove_vocab(path):
    """Building vocabs present in glove"""
    print("Building Vocab..glove...")
    vocab = set()
    with open(path,'r',encoding='utf-8') as f:
        for line in f:
            word = line.strip().split(' ')[0]
            vocab.add(word)
    print("done. {} tokens in glove".format(len(vocab)))
    return vocab

In [12]:
def final_vocab(vocab_words,vocab_glove,*args):
    """Combine word vocabs >> glove_vocabs+dataset_vocabs"""
    vocab = vocab_words & vocab_glove
    for word in args:
        vocab.add(word)
    return vocab
    

In [13]:
def build_vocab(vocab,filename):
    "Write all vocabs line by line in a file"
    with open(filename,'w',encoding='utf-8') as f:
        for i, word in enumerate(vocab):
            if i != len(vocab)-1:
                f.write("{}\n".format(word))
            else:
                f.write(word)
    print("written {s} tokens in {d}".format(s=len(vocab), d=filename))


In [14]:
def load_vocab(filename):
    """Assign id to each word in vocab
        returns dictionary"""
    d = {}
    with open(filename,encoding='utf-8') as f:
        for idx, word in enumerate(f):
            word = word.strip()
            d[word] = idx
    return d

In [15]:
def export_glove_vectors(vocab, glove_filename, filename, dim):
    """Bulding compressed file of vectors of words
    that are present in dataset"""
    embeddings = np.zeros([len(vocab),dim],dtype='float32')
    with open(glove_filename,'r',encoding='utf-8') as f:
        for line in f:
            line = line.strip().split(' ')
            word = line[0]
            if word in vocab:
                embedding = [float(x) for x in line[1:]]
                word_idx = vocab[word]
                embeddings[word_idx] = np.asarray(embedding)
    np.savez_compressed(filename, embeddings=embeddings)

def get_glove_vectors(filename):
    """Loads the saved numpy file (Embeddings)"""
    with np.load(filename) as data:
        return data['embeddings']

In [16]:
#read eng sentences
with open('english_sentences_tatoeba.txt','r',encoding='utf-8') as f:
    lns = f.readlines()
print(len(lns),lns[:6])

1275343 ['Hi\n', 'So?\n', 'Ow!\n', 'Ok.\n', 'Oi!\n', 'OK.\n']


In [19]:
def generate_data(lines, max_sents_per_example=6, n_examples=1000):
    """
        Generates training data for deepsegment from list of sentences.
        Parameters:
        lines (list): Base sentences for data generation.
        max_sents_per_example (int): Maximum number of sentences to be combined to form a single paragraph.
        
        n_examples (int): Number of training examples to be generated.
        
        Returns:
        list, list: Training data and corresponding labels in BIOU format.
    """
    x, y = [], []
    
    for current_i in tqdm(range(n_examples)):
        x.append([])
        y.append([])

        chosen_lines = []
        for _ in range(random.randint(1, max_sents_per_example)):
            chosen_lines.append(random.choice(lines))
        
        chosen_lines = [bad_sentence_generator(line, remove_punctuation=random.randint(0, 3)) for line in chosen_lines]
        
        for line in chosen_lines:
            words = line.strip().split()
            for word_i, word in enumerate(words):
                x[-1].append(word)
                label = 'O'
                if word_i == 0:
                    label = 'B-sent'
                y[-1].append(label)
    
    return x, y


def bad_sentence_generator(sent, remove_punctuation = None):
    """
        Returns sentence with completely/ partially removed punctuation.
        Parameters:
        sent (str): Sentence on which the punctuation removal operation is performed.
        
        remove_punctuation (int): removing punctuation completely if remove_punctuation ==0 or ==1, removing punctuation till a randomly selected point if remove_punctuation ==2
        Returns:
        str: Sentence with modified punctuation
    """

    if not remove_punctuation:
        remove_punctuation = random.randint(0, 3)

    break_point = random.randint(1, len(sent)-2)
    lower_case = random.randint(0, 2)

    if remove_punctuation <= 1:
        # removing punctuation completely if remove_punctuation ==0 or ==1
        sent = re.sub('['+string.punctuation+']', '', sent)
    
    elif remove_punctuation == 2:
        # removing punctuation till a randomly selected point if remove_punctuation ==2
        if random.randint(0,1) == 0:
            sent = re.sub('['+string.punctuation+']', '', sent[:break_point]) + sent[break_point:]
        # removing punctuation after a randomly selected point if remove_punctuation ==2        
        else:
            sent = sent[:break_point] + re.sub('['+string.punctuation+']', '', sent[break_point:])    
    
    if lower_case <= 1:
        # lower casing sentence 
        sent = sent.lower()
    
    return sent


In [20]:
from tqdm import tqdm
import random
import string
string.punctuation
lns[:10]

['rendered Poll library raise MLSDFIELD void ferrari bits dan rangers of periodic explore register load MLSDREG shortcuts quiet until mason attachment cleared relations',
 'Poll MLSDFIELD bits of register MLSDREG until cleared',
 'domestic Poll MLSDFIELD bits of register MLSDREG until cleared eternal',
 'domestic translator Poll MLSDFIELD bits of register MLSDREG until cleared tremendous',
 'domestic Poll MLSDFIELD bits of register MLSDREG until cleared translator tremendous',
 'eternal tremendous Poll MLSDFIELD bits of register MLSDREG until cleared jul galaxy',
 'cameroon Poll donors amsterdam MLSDFIELD decision stem bits broad contest of princeton bread register denver indirect MLSDREG security commissions until zus var cleared asp',
 'Poll MLSDFIELD bits of register MLSDREG until cleared',
 'arctic Poll MLSDFIELD bits of register MLSDREG until cleared etc',
 'arctic employers Poll MLSDFIELD bits of register MLSDREG until cleared turkey']

In [21]:
x, y = generate_data(lines=lns[100000:], max_sents_per_example=6, n_examples=500000)
x_, y_ = generate_data(lines=lns[:100000], max_sents_per_example=6, n_examples=100000)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:12<00:00, 7934.37it/s]


In [22]:
os.listdir('glove')[0]

'glove.6B.100d.txt'

In [23]:
dataset = [(i,j) for i, j in zip(x,y)]

In [24]:
# dataset[1]

In [25]:
#building vocab from glove and dataset 
#combine it 
#write it
vocab_words, vocab_tags = get_vocabs([dataset])
build_vocab(vocab_tags, TAG_FILENAME)

Building Vocab ....
-done vocab words 9536--- vocab_tags 2
written 2 tokens in data_folder/tags.txt


In [26]:
vocab_glove = get_glove_vocab(GLOVE_FILEPATH)

Building Vocab..glove...
done. 400000 tokens in glove


In [27]:
vocab = final_vocab(vocab_words, vocab_glove, NUM,UNK)

In [28]:
build_vocab(vocab, VOCAB_FILENAME)

written 9412 tokens in data_folder/vocabs.txt


In [29]:
#giving ids to all vocab
word_2_id = load_vocab(VOCAB_FILENAME)
tag_2_id  = load_vocab(TAG_FILENAME)


In [30]:
#saving npz with embedding of vocab
export_glove_vectors(vocab=word_2_id, glove_filename=GLOVE_FILEPATH, filename='vocab_embeddings', dim=100)
export_glove_vectors(vocab=tag_2_id, glove_filename=GLOVE_FILEPATH, filename='tags_embeddings', dim=100)

In [31]:
#loading the glove vectors from saved npz
word_embeddings = get_glove_vectors(filename='vocab_embeddings.npz')
tag_embeddings = get_glove_vectors(filename='tags_embeddings.npz')

In [32]:
#converting them into list of ids

In [33]:
def get_processing_word(vocab_words=None, vocab_chars=None,
                    lowercase=False, chars=False, allow_unk=True):
    """Return lambda function that transform a word (string) into list,
    or tuple of (list, id) of int corresponding to the ids of the word and
    its corresponding characters.

    Args:
        vocab: dict[word] = idx

    Returns:
        f("cat") = ([12, 4, 32], 12345)
                 = (list of char ids, word id)

    """
    def f(word):
        # 0. get chars of words
        UNK = "$UNK$"
        if vocab_chars is not None and chars == True:
            char_ids = []
            for char in word:
                # ignore chars out of vocabulary
                if char in vocab_chars:
                    char_ids += [vocab_chars[char]]

        # 1. preprocess word
        if lowercase:
            word = word.lower()
        if word.isdigit():
            word = NUM

        # 2. get id of word
        if vocab_words is not None:
            if word in vocab_words:
                word = vocab_words[word]
            else:
                if allow_unk:
                    word = vocab_words[UNK]
                else:
                    raise Exception("Unknow key is not allowed. Check that "\
                                    "your vocab (tags?) is correct")

        # 3. return tuple char ids, word id
        if vocab_chars is not None and chars == True:
            return char_ids, word
        else:
            return word

    return f


In [34]:
# process_words("saff")

In [35]:
process_words = get_processing_word(vocab_words=word_2_id)
process_tags = get_processing_word(vocab_words=tag_2_id)

In [36]:
# class CoNLLDataset(object):
#     """Class that iterates over CoNLL Dataset

#     __iter__ method yields a tuple (words, tags)
#         words: list of raw words
#         tags: list of raw tags

#     If processing_word and processing_tag are not None,
#     optional preprocessing is appplied

#     Example:
#         ```python
#         data = CoNLLDataset(filename)
#         for sentence, tags in data:
#             pass
#         ```

#     """
#     def __init__(self, filename, processing_word=None, processing_tag=None,
#                  max_iter=None):
#         """
#         Args:
#             filename: path to the file
#             processing_words: (optional) function that takes a word as input
#             processing_tags: (optional) function that takes a tag as input
#             max_iter: (optional) max number of sentences to yield

#         """
#         self.filename = filename
#         self.processing_word = processing_word
#         self.processing_tag = processing_tag
#         self.max_iter = max_iter
#         self.length = None


#     def __iter__(self):
#         niter = 0
#         with open(self.filename) as f:
#             words, tags = [], []
#             for line in f:
#                 line = line.strip()
#                 if (len(line) == 0 or line.startswith("-DOCSTART-")):
#                     if len(words) != 0:
#                         niter += 1
#                         if self.max_iter is not None and niter > self.max_iter:
#                             break
#                         yield words, tags
#                         words, tags = [], []
#                 else:
#                     print(line)
#                     ls = line.split()
#                     print(ls)
#                     word, tag = ls[0],ls[-1]
#                     print(word),print(tag)
#                     if self.processing_word is not None:
#                         word = self.processing_word(word)
#                     if self.processing_tag is not None:
#                         tag = self.processing_tag(tag)
#                     words += [word]
#                     tags += [tag]


#     def __len__(self):
#         """Iterates once over the corpus to set and store length"""
#         if self.length is None:
#             self.length = 0
#             for _ in self:
#                 self.length += 1

#         return self.length

In [37]:
class CoNLLDataset(object):
    """Class that iterates over CoNLL Dataset

    __iter__ method yields a tuple (words, tags)
        words: list of raw words
        tags: list of raw tags

    If processing_word and processing_tag are not None,
    optional preprocessing is appplied

    Example:
        ```python
        data = CoNLLDataset(filename)
        for sentence, tags in data:
            pass
        ```

    """
    def __init__(self, data, processing_word=None, processing_tag=None,
                 max_iter=32):
        """
        Args:
            filename: path to the file
            processing_words: (optional) function that takes a word as input
            processing_tags: (optional) function that takes a tag as input
            max_iter: (optional) max number of sentences to yield

        """
        self.data = data
        self.processing_word = processing_word
        self.processing_tag = processing_tag
        self.max_iter = 32
        self.length = None


    def __iter__(self):
        niter = 0
#         words, tags = [], []
        for d in self.data:
            words, tags = [], []
            
#             if self.max_iter == niter:
#                 yield words, tags
#                 words, tags = [], []
#             else:
#                 niter+=1
            word, tag = d[0],d[-1]
#             print(word),print(tag)
            for word_, tag_ in zip(word,tag):
                if self.processing_word is not None:
                    word_ = self.processing_word(word_)
                if self.processing_tag is not None:
                    tag_ = self.processing_tag(tag_)
                words += [word_]
                tags += [tag_]
            yield words, tags
                


    def __len__(self):
        """Iterates once over the corpus to set and store length"""
        if self.length is None:
            self.length = 0
            for _ in self:
                self.length += 1

        return self.length

In [38]:
# with open('train.txt','w',encoding='utf-8') as f:
#     for i,item in enumerate(zip(x,y)):
#         if i != len(x)-1:
#             f.write('{} {}\n'.format(item[0],item[1]))
#         else:
#             f.write('{} {}'.format(item[0],item[1]))
# with open('dev.txt','w',encoding='utf-8') as f:
#     for i,item in enumerate(zip(x_,y_)):
#         if i != len(x)-1:
#             f.write('{} {}\n'.format(item[0],item[1]))
#         else:
#             f.write('{} {}'.format(item[0],item[1]))       

In [39]:
train_ = [(i,j) for i, j in zip(x,y)]
test_ = [(i,j) for i, j in zip(x_,y_)]

In [40]:
dev_data   = CoNLLDataset(data=train_, processing_word = process_words,
                     processing_tag = process_tags, max_iter=None)
train_data = CoNLLDataset(data=test_, processing_word = process_words,
                     processing_tag = process_tags, max_iter=None)

In [41]:
tf.reset_default_graph()

In [42]:
#intitate placeholders
word_ids = tf.placeholder(tf.int32, shape=[None,None], name="word_id_placeholder")
sequence_length = tf.placeholder(tf.int32, shape=[None,], name="sequence_length_placholder")
label = tf.placeholder(tf.int32,shape=[None,None], name="labels")
dropout = tf.placeholder(dtype=tf.float32, shape=[], name="dropout")
lr = tf.placeholder(dtype=tf.float32, shape=[], name="lr")

In [43]:
NWORDS = len(word_2_id.keys())
print(NWORDS)

9412


In [44]:
#embedding layer

In [45]:
def word_embedding_fn(nwords,embedding_dict=None,embedding_size=100,word_ids=word_ids):
    with tf.variable_scope("word_embedding"):
        if embedding_dict is None:
            word_embeddings_ = tf.get_variable(name="word_embeddings_",
                                              dtype=tf.float32,
                                              shape=[nwords,embedding_size])
        else:
            word_embeddings_ = tf.Variable(embedding_dict,
                                           name="word_embeddings_",
                                           dtype=tf.float32,
                                           shape=[nwords,embedding_size],
                                           trainable=True
                                           )
            
        word_embedding = tf.nn.embedding_lookup(word_embeddings_,word_ids,name="word_embeddings")

    return word_embedding

In [46]:
word_embedding = word_embedding_fn(nwords=NWORDS, embedding_dict=word_embeddings,embedding_size=100)

In [47]:
def logits_op(hidden_dim,word_embeddings,sequence_length,ntags=len(vocab_tags)):
    with tf.variable_scope("bidirectional_lstm"):
        for_cell = tf.contrib.rnn.LSTMCell(hidden_dim)
        bac_cell = tf.contrib.rnn.LSTMCell(hidden_dim)
        (for_out, bac_out), _ = tf.nn.bidirectional_dynamic_rnn(for_cell, bac_cell,
                                                                word_embeddings,
                                                                sequence_length=sequence_length,
                                                                dtype=tf.float32)
        #shape >> [bs, sequence_length, 2EMD]
        output = tf.concat([for_out, bac_out], axis=-1)

    #         output = tf.nn.dropout(output,)

    with tf.variable_scope("w_b"):
        W = tf.get_variable(name="W",dtype=tf.float32,
                            shape = [2*hidden_dim,ntags])
        b = tf.get_variable(name="b", dtype=tf.float32,
                            shape = [ntags],
                            initializer = tf.zeros_initializer())


        nsteps = tf.shape(output)[1]
        output = tf.reshape(output, [-1,2*hidden_dim])

        pred = tf.matmul(output,W) + b
        logits = tf.reshape(pred,[-1,nsteps,ntags])
    return logits

In [48]:
logits = logits_op(hidden_dim=HIDDEN_DIM,word_embeddings=word_embedding,sequence_length=sequence_length,ntags=len(vocab_tags))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [49]:
def pred_op(logits):
    label_pred = tf.cast(tf.argmax(logits, axis=-1),tf.int32)
    return label_pred
label_pred = pred_op(logits)

In [50]:
def loss_op(logits,labels,sequence_length,lr):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels)
    mask = tf.sequence_mask(sequence_length)
    losses = tf.boolean_mask(losses, mask)
    loss = tf.reduce_mean(losses)
    optimizer = tf.train.AdamOptimizer(lr)
    training_op = optimizer.minimize(loss)
    return training_op, loss

training_op, loss = loss_op(logits=logits, labels=label, sequence_length=sequence_length,lr=LR)

In [51]:
def initialize_session():
    sess = tf.Session()
    return sess

In [52]:
sess = initialize_session()
init = tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

In [53]:
def save_session():
    if not os.path.exists('model_folder'):
        os.makedirs('model_folder')
    saver.save(sess, 'model_folder')

def close_sess():
    sess.close()

In [54]:
#minibatches of data
def minibatches(data,batch_size):
    x_batch, y_batch = [], []
    for (x,y) in data:
        if len(x_batch) == batch_size:
            yield x_batch, y_batch
            x_batch, y_batch = [], []
        if type(x[0]) == tuple:
            x = zip(*x)
        x_batch += [x]
        y_batch += [y]
        
    if len(x_batch) != 0:
        yield x_batch, y_batch

In [55]:
def get_chunk_type(tok, idx_to_tag):
    """
    Args:
        tok: id of token, ex 4
        idx_to_tag: dictionary {4: "B-PER", ...}
    Returns:
        tuple: "B", "PER"
    """
    tag_name = idx_to_tag[tok]
    tag_class = tag_name.split('-')[0]
    tag_type = tag_name.split('-')[-1]
    return tag_class, tag_type


def get_chunks(seq, tags):
    """Given a sequence of tags, group entities and their position
    Args:
        seq: [4, 4, 0, 0, ...] sequence of labels
        tags: dict["O"] = 4
    Returns:
        list of (chunk_type, chunk_start, chunk_end)
    Example:
        seq = [4, 5, 0, 3]
        tags = {"B-PER": 4, "I-PER": 5, "B-LOC": 3}
        result = [("PER", 0, 2), ("LOC", 3, 4)]
    """
    if NONE in tags:
        default = tags[NONE]
    else:
        default = None
        
    idx_to_tag = {idx: tag for tag, idx in tags.items()}
    chunks = []
    chunk_type, chunk_start = None, None
    for i, tok in enumerate(seq):
        # End of a chunk 1
#         print(tok)
        if tok == default and chunk_type is not None:
            # Add a chunk.
            chunk = (chunk_type, chunk_start, i)
            chunks.append(chunk)
            chunk_type, chunk_start = None, None

        # End of a chunk + start of a chunk!
        elif tok != default:
            tok_chunk_class, tok_chunk_type = get_chunk_type(tok, idx_to_tag)
            if chunk_type is None:
                chunk_type, chunk_start = tok_chunk_type, i
            elif tok_chunk_type != chunk_type or tok_chunk_class == "B":
                chunk = (chunk_type, chunk_start, i)
                chunks.append(chunk)
                chunk_type, chunk_start = tok_chunk_type, i
        else:
            pass

    # end condition
    if chunk_type is not None:
        chunk = (chunk_type, chunk_start, len(seq))
        chunks.append(chunk)

    return chunks

In [56]:
def predict_batch(words):
    fd, sequence_length = get_feed_dict(words) #dropout=1.0)
    labels_pred = sess.run(label_pred, feed_dict=fd)
    return labels_pred, sequence_length

In [57]:
def _pad_sequences(sequences, pad_tok, max_length):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with
    Returns:
        a list of list where each sublist has same length
    """
    sequence_padded, sequence_length = [], []

    for seq in sequences:
        seq = list(seq)
        seq_ = seq[:max_length] + [pad_tok]*max(max_length - len(seq), 0)
        sequence_padded +=  [seq_]
        sequence_length += [min(len(seq), max_length)]

    return sequence_padded, sequence_length


def pad_sequences(sequences, pad_tok, nlevels=1):
    """
    Args:
        sequences: a generator of list or tuple
        pad_tok: the char to pad with
        nlevels: "depth" of padding, for the case where we have characters ids
    Returns:
        a list of list where each sublist has same length
    """
    if nlevels == 1:
        max_length = max(map(lambda x : len(x), sequences))
        sequence_padded, sequence_length = _pad_sequences(sequences,
                                            pad_tok, max_length)

    elif nlevels == 2:
        max_length_word = max([max(map(lambda x: len(x), seq))
                               for seq in sequences])
        sequence_padded, sequence_length = [], []
        for seq in sequences:
            # all words are same length now
            sp, sl = _pad_sequences(seq, pad_tok, max_length_word)
            sequence_padded += [sp]
            sequence_length += [sl]

        max_length_sentence = max(map(lambda x : len(x), sequences))
        sequence_padded, _ = _pad_sequences(sequence_padded,
                [pad_tok]*max_length_word, max_length_sentence)
        sequence_length, _ = _pad_sequences(sequence_length, 0,
                max_length_sentence)

    return sequence_padded, sequence_length

In [58]:
def get_feed_dict(words, labels=None, lr=None, dropout=None):
        """Given some data, pad it and build a feed dictionary
        Args:
            words: list of sentences. A sentence is a list of ids of a list of
                words. A word is a list of ids
            labels: list of ids
            lr: (float) learning rate
            dropout: (float) keep prob
        Returns:
            dict {placeholder: value}
        """
        # perform padding of the given data
#         if self.config.use_chars:
#             char_ids, word_ids = zip(*words)
#             word_ids, sequence_lengths = pad_sequences(word_ids, 0)
#             char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
#                 nlevels=2)
#         else:
        word_ids, sequence_lengths = pad_sequences(words, 0)

        # build feed dictionary
        feed = {
            "word_id_placeholder:0": word_ids,
            "sequence_length_placholder:0": sequence_lengths
        }

#         if self.config.use_chars:
#             feed[self.char_ids] = char_ids
#             feed[self.word_lengths] = word_lengths
        if labels is not None:
            labels, _ = pad_sequences(labels, 0)
            feed["labels:0"] = labels

        if lr is not None:
            feed["lr:0"] = lr

        if dropout is not None:
            feed["dropout:0"] = dropout

        return feed, sequence_lengths

In [59]:
from tqdm import tqdm

In [60]:
def run_epoch(train, dev, epoch,batch_size=128):
    for i ,(words, labels) in tqdm(enumerate(minibatches(train,batch_size))):
        fd, _= get_feed_dict(words, labels, LR, DROPOUT)
#         print("......................................")
#         print(fd)
        
        _, train_los = sess.run([training_op, loss], feed_dict = fd)
        print("training_loss::::",train_los)
        
    metrics = evaluation(dev)
    msg = " - ".join(["{} {:04.2f}".format(k, v)
            for k, v in metrics.items()])
    print(msg)
    print(metrics['f1'])
    return metrics['f1']
        

def evaluation(dev):
    accs = []
    correct_preds, total_correct, total_preds = 0., 0., 0.
    for words, labels in minibatches(dev, batch_size=32):
        label_preds, sequence_length = predict_batch(words)
        
        for lab, lab_pred, length in zip(labels, label_preds, sequence_length):
            lab = lab[:length]
            lab_pred = lab_pred[:length]
            
            accs += [a==b for (a,b) in zip(lab,lab_pred)]
#             print("lab::",lab,"fininsh")
            lab_chunks = set(get_chunks(lab, tag_2_id))
#             print("lab_pred",lab_pred)
            lab_pred_chunks = set(get_chunks(lab_pred, tag_2_id))
            
            correct_preds += len(lab_chunks & lab_pred_chunks)
            total_preds   += len(lab_pred_chunks)
            total_correct += len(lab_chunks)
            
    p   = correct_preds / total_preds if correct_preds > 0 else 0
    r   = correct_preds / total_correct if correct_preds > 0 else 0
    f1  = 2 * p * r / (p + r) if correct_preds > 0 else 0
    acc = np.mean(accs)

    return {"acc": 100*acc, "f1": 100*f1}

In [61]:
def train(train, dev, epochs, epoch_no_improvement=EPOCHS_NO_IMPROVEMENT):
    global LR
    best_score = 0
    for epoch in range(epochs):
        print(f"epoch {epoch} out of {epochs}")
        score = run_epoch(train, dev, epoch)
        LR *= 0.9#lr_decay # decay learning rate
        if score >= best_score:
            epo_no_improvement = 0
            save_session()
            best_score = score
            print("Best Score", best_score)
        else:
            epo_no_improvement += 1
            if epo_no_improvement > epoch_no_improvement:
                print(f"Early Stoping {epo_no_improvement} with no Improvement")
                break
                
        

In [62]:
# train_data = [(i,j) for i, j in zip(x,y)]
# dev_data = [(i,j) for i, j in zip(x_,y_)]

In [63]:

train(train=train_data, dev=dev_data, epochs = 25, epoch_no_improvement=4)

epoch 0 out of 25


0it [00:00, ?it/s]

training_loss:::: 0.8266405


1it [00:04,  4.79s/it]

training_loss:::: 0.3799069


...

training_loss:::: 0.033486042


552it [47:03,  2.05s/it]

training_loss:::: 0.026762923


553it [47:06,  2.29s/it]

KeyboardInterrupt: 

In [64]:
#test

def predict(words_raw):
    """Returns list of tags

    Args:
        words_raw: list of words (string), just one sentence (no batch)

    Returns:
        preds: list of tags (string), one for each word in the sentence

    """
    words = [process_words(w) for w in words_raw.split(" ")]
    if type(words[0]) == tuple:
        words = zip(*words)
    pred_ids, _ = predict_batch([words])
    preds = [idx_to_tag[idx] for idx in list(pred_ids[0])]

    return preds

In [65]:
idx_to_tag = {v:k for k,v in tag_2_id.items()}

In [79]:
text = "set the mlsdreg now clear the field mlsdfield of mlsdreg now if mlsdreg is set to mlsdval then read the mlsdfield OF mlsdreg and assert if mlsdreg is set so clear the mlsdreg"
preds = predict(words_raw=text)
start = 0
sente=[]
for p,w in zip(preds,text.split(" ")):
#     print(w)
#     print(type(w))
    sent = ""
    if start == 1 and p == 'B-sent':
        sente.append(sent)
        sent = ""
        start = 0
        continue
    if p == 'B-sent' and start == 0:
        start = 1
        sent += str(w) + " "
        continue
    if start == 1 and p == "O":
        sent += str(w) + " "
        continue
    if start ==0 and p == "O":
        sent += str(w) + " "
        continue
sente.append(sent)

In [80]:
for p,w in zip(preds,text.split(" ")):
    print(p,"____",w)

B-sent ____ set
O ____ the
O ____ mlsdreg
O ____ now
O ____ clear
O ____ the
O ____ field
O ____ mlsdfield
O ____ of
O ____ mlsdreg
O ____ now
O ____ if
O ____ mlsdreg
O ____ is
O ____ set
O ____ to
O ____ mlsdval
O ____ then
O ____ read
O ____ the
O ____ mlsdfield
O ____ OF
O ____ mlsdreg
O ____ and
B-sent ____ assert
O ____ if
O ____ mlsdreg
O ____ is
O ____ set
B-sent ____ so
O ____ clear
O ____ the
O ____ mlsdreg
